In [182]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Work in Progress

# Decision with 2 Objectives

[Link](https://dmcommunity.org/challenge/challenge-oct-2025/) to the original post.

This challenge is offered by Dr. Meinolf Sellmann. A freelance webpage developer received a task. A client has a budget of $10,000 and wants a webpage developed with as many features as possible, but also maximizing the total value of these features:

![](https://dmcommunity.org/wp-content/uploads/2025/10/image-6.png)

The client leaves the decision of which features to the developer. She wants to delight the client and asks you which features would be best to select. We have the budget constraint and two objectives. Can you devise a rational way to trade them against each other?

In [183]:
budget = 10_000
features = {
    1: {"Cost": 5000, "Value": 7},
    2: {"Cost": 4000, "Value": 6},
    3: {"Cost": 3000, "Value": 5},
    4: {"Cost": 2000, "Value": 4},
    5: {"Cost": 1000, "Value": 3},
    6: {"Cost": 1000, "Value": 2},
    7: {"Cost": 1000, "Value": 1},
    8: {"Cost": 1000, "Value": 1},
    9: {"Cost": 1000, "Value": 1},
    10: {"Cost": 1000, "Value": 1}
}

In [184]:
model = ConcreteModel()

# Boolean variable for which features to choose
model.u = Var(list(features.keys()), within = Boolean)

# Must not go over the budget
model.Budget = Constraint(expr = sum(model.u[i]*features[i]['Cost'] for i in model.u) <= budget)

# Maximize Value and features chosen
model.obj = Objective(expr = sum(model.u[i] + model.u[i]*features[i]["Value"] for i in model.u), sense = maximize)

In [185]:
# Solve model
opt = SolverFactory('glpk')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

soln = model.u.extract_values()

print(f'Features chosen: {[i for  i in model.u if soln[i] == 1]}')
print(f'Value: {sum(soln[i]*features[i]["Value"] for i in model.u)}')
print(f'Cost: {value(model.Budget)}')

Solution is Optimal
Solve Time:  <undefined>
Features chosen: [3, 4, 5, 6, 8, 9, 10]
Value: 17.0
Cost: 10000.0


# Advanced Website Design

[Link](https://dmcommunity.org/challenge-nov-2025/) to the original post.

Now you have several additional requirements:

- Features 3 and 4 can be chosen only together.
- Feature 2 cannot be combined with Feature 3.
- Only one of the features 8, 9, and 10 can be selected. Otherwise, the cost of each of these features will be increased by 10%.
- If 5 or more features are selected, 5% discount is provided.

You still want to offer a design that maximizes the total value while minimizing the total cost.

In [186]:
model = ConcreteModel()

# Boolean variable for which features to choose
model.u = Var(list(features.keys()), within = Boolean)

# Features 3 and 4 can be chosen only together
model.con2 = Constraint(expr = model.u[3] == model.u[4])

# Feature 2 cannot be combined with Feature 3.
model.con3 = Constraint(expr = model.u[2] == 1 - model.u[3])

# Only one of the features 8, 9, and 10 can be selected. Otherwise, the cost of each of these features will be increased by 10%.
model.incr_val = Var(bounds = (0, 3), within = Integers)
model.x = Var(within = Boolean)

model.con4 = Constraint(expr = sum(model.u[i] for i in [8,9,10]) == model.incr_val)
model.con5 = Constraint(expr= model.incr_val <= 1 + 2*model.x)
model.con6 = Constraint(expr= model.incr_val >= 2*model.x)

# New cost to support 10% increase
model.cost = Var(list(features.keys()), bounds = (0,50000), within = PositiveReals)

def cost_rule_inc(model, i):
    if i in [8,9,10]:
        return model.cost[i] == features[i]['Cost']*model.x*1.1 + features[i]['Cost']*(1-model.x)
    else:
        return model.cost[i] == features[i]['Cost']
model.con10 = Constraint(list(features.keys()), rule=cost_rule_inc)

# If 5 or more features are selected, 5% discount is provided.
model.disc_val = Var(bounds = (0, 5), within = Integers)
model.z = Var(within = Boolean)

model.con7 = Constraint(expr = sum(model.u[i] for i in model.u) == model.disc_val)
model.con8 = Constraint(expr= model.disc_val <= 4 + 5*model.z)
model.con9 = Constraint(expr= model.disc_val >= 5*model.z)

# model.test = Constraint(expr = sum(model.u[i] for i in [3,4,5,6,7,8]) == 6)

# Must not go over the budget
model.Budget = Constraint(expr = sum(model.u[i]*model.cost[i]*model.z*.95 + model.u[i]*model.cost[i]*(1-model.z) for i in model.u) <= budget)

# model.test = Constraint(expr = sum(model.u[i]*features[i]['Value'] for i in model.u) == 16)

model.obj = Objective(expr = sum(model.u[i] for i in model.u) + \
                      sum(model.u[i]*features[i]['Value'] for i in model.u), sense = maximize)

In [209]:
# Solve model
opt = SolverFactory('ipopt')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

soln = model.u.extract_values()

print(f'Features chosen: {[i for  i in model.u if soln[i] == 1]}')
print(f'Value: {sum(soln[i]*features[i]["Value"] for i in model.u)}')
print(f'Cost: {value(model.Budget)}')


Solution is Optimal
Solve Time:  <undefined>
Features chosen: [3, 4, 5, 6]
Value: 18.789473796934466
Cost: 10000.000079501719
